In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#!zip -F file.zip --out file-large.zip
#!unzip file-large.zip
#!unzip "/content/drive/My Drive/img_align_celeba.zip"
!cat '/content/drive/My Drive/utils.py'
!cat '/content/drive/My Drive/ops.py'
!cat '/content/drive/My Drive/models_64x64.py'
import sys
import imageio
sys.path.append('/content/drive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import re
import scipy
import numpy as np
import tensorflow as tf
import cv2
import imageio

from collections import OrderedDict


def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        path_dir, _ = os.path.split(path)
        if not os.path.isdir(path_dir):
            os.makedi

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models


""" param """
epoch = 50
batch_size = 64
lr = 0.0002
z_dim = 100
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]


def preprocess_fn(img):
    crop_size = 108
    re_size = 64
    img = tf.image.crop_to_bounding_box(img, (218 - crop_size) // 2, (178 - crop_size) // 2, crop_size, crop_size)
    img = tf.to_float(tf.image.resize_images(img, [re_size, re_size], method=tf.image.ResizeMethod.BICUBIC)) / 127.5 - 1
    return img

img_paths = glob.glob('/content/img_align_celeba/')
data_pool = utils.DiskImageData('/content/img_align_celeba/', batch_size, shape=[218, 178, 3], preprocess_fn=preprocess_fn)

tf.reset_default_graph()

""" graphs """
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator

    ''' graph '''
    # inputs
    real = tf.placeholder(tf.float32, shape=[None, 64, 64, 3])
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, reuse=False)

    # dicriminate
    r_logit = discriminator(real, reuse=False)
    f_logit = discriminator(fake)

    # losses
    d_r_loss = tf.losses.sigmoid_cross_entropy(tf.ones_like(r_logit), r_logit)
    d_f_loss = tf.losses.sigmoid_cross_entropy(tf.zeros_like(f_logit), f_logit)
    d_loss = (d_r_loss + d_f_loss) / 2.0
    g_loss = tf.losses.sigmoid_cross_entropy(tf.ones_like(f_logit), f_logit)

    # otpims
    d_var = utils.trainable_variables('discriminator')
    g_var = utils.trainable_variables('generator')
    d_step = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5).minimize(d_loss, var_list=d_var)
    g_step = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5).minimize(g_loss, var_list=g_var)

    # summaries
    d_summary = utils.summary({d_loss: 'd_loss'})
    g_summary = utils.summary({g_loss: 'g_loss'})

    # sample
    f_sample = generator(z, training=False)
    f_sample2 = tf.cast(tf.reshape(f_sample[0,:,:,:]*255, [1,64,64,3]), tf.uint8)
    import pdb
    # pdb.set_trace()
    f_summary = utils.summary({f_sample2: 'image'}, summary_type='image')

""" train """
''' init '''
# session
sess = utils.session()
# iteration counter
it_cnt, update_cnt = utils.counter()
# saver
saver = tf.train.Saver(max_to_keep=5)
# summary writer
summary_writer = tf.summary.FileWriter('/content/drive/My Drive/GP/summaries/celeba_dcgan', sess.graph)

ckpt_dir = '/content/drive/My Drive/GP/checkpoints/celeba_dcgan'
utils.mkdir(ckpt_dir + '/')
#if not utils.load_checkpoint(ckpt_dir, sess):
    #sess.run(tf.global_variables_initializer())
''' train '''
try:
    ckpt_file = '/content/drive/My Drive/GP/checkpoints/celeba_dcgan/Epoch_(45)_(1575of3165).ckpt'
    saver.restore(sess, ckpt_file)
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, ckpt_file)

    z_ipt_sample = np.random.normal(size=[100, z_dim])

    batch_epoch = len(data_pool) // (batch_size)
    max_it = epoch * batch_epoch
    for it in range(sess.run(it_cnt), max_it):
        sess.run(update_cnt)

        # which epoch
        epoch = it // batch_epoch
        it_epoch = it % batch_epoch + 1

        # batch data
        real_ipt = data_pool.batch()
        z_ipt = np.random.normal(size=[batch_size, z_dim])

        # train D
        d_summary_opt, _ = sess.run([d_summary, d_step], feed_dict={real: real_ipt, z: z_ipt})
        summary_writer.add_summary(d_summary_opt, it)

        # train G
        sess.run([g_step], feed_dict={z: z_ipt})
        g_summary_opt, _ = sess.run([g_summary, g_step], feed_dict={z: z_ipt})
        summary_writer.add_summary(g_summary_opt, it)

        # display
        if it % 1 == 0:
            print("Epoch: (%3d) (%5d/%5d)" % (epoch, it_epoch, batch_epoch))

        # save
        if (it + 1) % 1000 == 0:
            save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_epoch, batch_epoch))
            print('Model saved in file: % s' % save_path)

        # sample
        if (it + 1) % 100 == 0:
            f_sample_opt = sess.run(f_sample, feed_dict={z: z_ipt_sample})
           # f_summary_opt, _ = sess.run([f_summary, it], feed_dict={z: z_ipt_sample})
           # summary_writer.add_summary(f_summary_opt, it)

            save_dir = '/content/drive/My Drive/GP/sample_images_while_training/celeba_dcgan'
            utils.mkdir(save_dir + '/')
            utils.imwrite(utils.immerge(f_sample_opt, 10, 10), '%s/Epoch_(%d)_(%dof%d).jpg' % (save_dir, epoch, it_epoch, batch_epoch))

except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

#Data is  202599
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuf

Epoch: ( 45) ( 1676/ 3165)
Epoch: ( 45) ( 1677/ 3165)
Epoch: ( 45) ( 1678/ 3165)
Epoch: ( 45) ( 1679/ 3165)
Epoch: ( 45) ( 1680/ 3165)
Epoch: ( 45) ( 1681/ 3165)
Epoch: ( 45) ( 1682/ 3165)
Epoch: ( 45) ( 1683/ 3165)
Epoch: ( 45) ( 1684/ 3165)
Epoch: ( 45) ( 1685/ 3165)
Epoch: ( 45) ( 1686/ 3165)
Epoch: ( 45) ( 1687/ 3165)
Epoch: ( 45) ( 1688/ 3165)
Epoch: ( 45) ( 1689/ 3165)
Epoch: ( 45) ( 1690/ 3165)
Epoch: ( 45) ( 1691/ 3165)
Epoch: ( 45) ( 1692/ 3165)
Epoch: ( 45) ( 1693/ 3165)
Epoch: ( 45) ( 1694/ 3165)
Epoch: ( 45) ( 1695/ 3165)
Epoch: ( 45) ( 1696/ 3165)
Epoch: ( 45) ( 1697/ 3165)
Epoch: ( 45) ( 1698/ 3165)
Epoch: ( 45) ( 1699/ 3165)
Epoch: ( 45) ( 1700/ 3165)
Epoch: ( 45) ( 1701/ 3165)
Epoch: ( 45) ( 1702/ 3165)
Epoch: ( 45) ( 1703/ 3165)
Epoch: ( 45) ( 1704/ 3165)
Epoch: ( 45) ( 1705/ 3165)
Epoch: ( 45) ( 1706/ 3165)
Epoch: ( 45) ( 1707/ 3165)
Epoch: ( 45) ( 1708/ 3165)
Epoch: ( 45) ( 1709/ 3165)
Epoch: ( 45) ( 1710/ 3165)
Epoch: ( 45) ( 1711/ 3165)
Epoch: ( 45) ( 1712/ 3165)
E

Epoch: ( 45) ( 1776/ 3165)
Epoch: ( 45) ( 1777/ 3165)
Epoch: ( 45) ( 1778/ 3165)
Epoch: ( 45) ( 1779/ 3165)
Epoch: ( 45) ( 1780/ 3165)
Epoch: ( 45) ( 1781/ 3165)
Epoch: ( 45) ( 1782/ 3165)
Epoch: ( 45) ( 1783/ 3165)
Epoch: ( 45) ( 1784/ 3165)
Epoch: ( 45) ( 1785/ 3165)
Epoch: ( 45) ( 1786/ 3165)
Epoch: ( 45) ( 1787/ 3165)
Epoch: ( 45) ( 1788/ 3165)
Epoch: ( 45) ( 1789/ 3165)
Epoch: ( 45) ( 1790/ 3165)
Epoch: ( 45) ( 1791/ 3165)
Epoch: ( 45) ( 1792/ 3165)
Epoch: ( 45) ( 1793/ 3165)
Epoch: ( 45) ( 1794/ 3165)
Epoch: ( 45) ( 1795/ 3165)
Epoch: ( 45) ( 1796/ 3165)
Epoch: ( 45) ( 1797/ 3165)
Epoch: ( 45) ( 1798/ 3165)
Epoch: ( 45) ( 1799/ 3165)
Epoch: ( 45) ( 1800/ 3165)
Epoch: ( 45) ( 1801/ 3165)
Epoch: ( 45) ( 1802/ 3165)
Epoch: ( 45) ( 1803/ 3165)
Epoch: ( 45) ( 1804/ 3165)
Epoch: ( 45) ( 1805/ 3165)
Epoch: ( 45) ( 1806/ 3165)
Epoch: ( 45) ( 1807/ 3165)
Epoch: ( 45) ( 1808/ 3165)
Epoch: ( 45) ( 1809/ 3165)
Epoch: ( 45) ( 1810/ 3165)
Epoch: ( 45) ( 1811/ 3165)
Epoch: ( 45) ( 1812/ 3165)
E

Epoch: ( 45) ( 1876/ 3165)
Epoch: ( 45) ( 1877/ 3165)
Epoch: ( 45) ( 1878/ 3165)
Epoch: ( 45) ( 1879/ 3165)
Epoch: ( 45) ( 1880/ 3165)
Epoch: ( 45) ( 1881/ 3165)
Epoch: ( 45) ( 1882/ 3165)
Epoch: ( 45) ( 1883/ 3165)
Epoch: ( 45) ( 1884/ 3165)
Epoch: ( 45) ( 1885/ 3165)
Epoch: ( 45) ( 1886/ 3165)
Epoch: ( 45) ( 1887/ 3165)
Epoch: ( 45) ( 1888/ 3165)
Epoch: ( 45) ( 1889/ 3165)
Epoch: ( 45) ( 1890/ 3165)
Epoch: ( 45) ( 1891/ 3165)
Epoch: ( 45) ( 1892/ 3165)
Epoch: ( 45) ( 1893/ 3165)
Epoch: ( 45) ( 1894/ 3165)
Epoch: ( 45) ( 1895/ 3165)
Epoch: ( 45) ( 1896/ 3165)
Epoch: ( 45) ( 1897/ 3165)
Epoch: ( 45) ( 1898/ 3165)
Epoch: ( 45) ( 1899/ 3165)
Epoch: ( 45) ( 1900/ 3165)
Epoch: ( 45) ( 1901/ 3165)
Epoch: ( 45) ( 1902/ 3165)
Epoch: ( 45) ( 1903/ 3165)
Epoch: ( 45) ( 1904/ 3165)
Epoch: ( 45) ( 1905/ 3165)
Epoch: ( 45) ( 1906/ 3165)
Epoch: ( 45) ( 1907/ 3165)
Epoch: ( 45) ( 1908/ 3165)
Epoch: ( 45) ( 1909/ 3165)
Epoch: ( 45) ( 1910/ 3165)
Epoch: ( 45) ( 1911/ 3165)
Epoch: ( 45) ( 1912/ 3165)
E

Epoch: ( 45) ( 1976/ 3165)
Epoch: ( 45) ( 1977/ 3165)
Epoch: ( 45) ( 1978/ 3165)
Epoch: ( 45) ( 1979/ 3165)
Epoch: ( 45) ( 1980/ 3165)
Epoch: ( 45) ( 1981/ 3165)
Epoch: ( 45) ( 1982/ 3165)
Epoch: ( 45) ( 1983/ 3165)
Epoch: ( 45) ( 1984/ 3165)
Epoch: ( 45) ( 1985/ 3165)
Epoch: ( 45) ( 1986/ 3165)
Epoch: ( 45) ( 1987/ 3165)
Epoch: ( 45) ( 1988/ 3165)
Epoch: ( 45) ( 1989/ 3165)
Epoch: ( 45) ( 1990/ 3165)
Epoch: ( 45) ( 1991/ 3165)
Epoch: ( 45) ( 1992/ 3165)
Epoch: ( 45) ( 1993/ 3165)
Epoch: ( 45) ( 1994/ 3165)
Epoch: ( 45) ( 1995/ 3165)
Epoch: ( 45) ( 1996/ 3165)
Epoch: ( 45) ( 1997/ 3165)
Epoch: ( 45) ( 1998/ 3165)
Epoch: ( 45) ( 1999/ 3165)
Epoch: ( 45) ( 2000/ 3165)
Epoch: ( 45) ( 2001/ 3165)
Epoch: ( 45) ( 2002/ 3165)
Epoch: ( 45) ( 2003/ 3165)
Epoch: ( 45) ( 2004/ 3165)
Epoch: ( 45) ( 2005/ 3165)
Epoch: ( 45) ( 2006/ 3165)
Epoch: ( 45) ( 2007/ 3165)
Epoch: ( 45) ( 2008/ 3165)
Epoch: ( 45) ( 2009/ 3165)
Epoch: ( 45) ( 2010/ 3165)
Epoch: ( 45) ( 2011/ 3165)
Epoch: ( 45) ( 2012/ 3165)
E

...

Epoch: ( 49) (  615/ 3165)
Epoch: ( 49) (  616/ 3165)
Epoch: ( 49) (  617/ 3165)
Epoch: ( 49) (  618/ 3165)
Epoch: ( 49) (  619/ 3165)
Epoch: ( 49) (  620/ 3165)
Epoch: ( 49) (  621/ 3165)
Epoch: ( 49) (  622/ 3165)
Epoch: ( 49) (  623/ 3165)
Epoch: ( 49) (  624/ 3165)
Epoch: ( 49) (  625/ 3165)
Epoch: ( 49) (  626/ 3165)
Epoch: ( 49) (  627/ 3165)
Epoch: ( 49) (  628/ 3165)
Epoch: ( 49) (  629/ 3165)
Epoch: ( 49) (  630/ 3165)
Epoch: ( 49) (  631/ 3165)
Epoch: ( 49) (  632/ 3165)
Epoch: ( 49) (  633/ 3165)
Epoch: ( 49) (  634/ 3165)
Epoch: ( 49) (  635/ 3165)
Epoch: ( 49) (  636/ 3165)
Epoch: ( 49) (  637/ 3165)
Epoch: ( 49) (  638/ 3165)
Epoch: ( 49) (  639/ 3165)
Epoch: ( 49) (  640/ 3165)
Epoch: ( 49) (  641/ 3165)
Epoch: ( 49) (  642/ 3165)
Epoch: ( 49) (  643/ 3165)
Epoch: ( 49) (  644/ 3165)
Epoch: ( 49) (  645/ 3165)
Epoch: ( 49) (  646/ 3165)
Epoch: ( 49) (  647/ 3165)
Epoch: ( 49) (  648/ 3165)
Epoch: ( 49) (  649/ 3165)
Epoch: ( 49) (  650/ 3165)
Epoch: ( 49) (  651/ 3165)
E

Epoch: ( 49) (  715/ 3165)
Epoch: ( 49) (  716/ 3165)
Epoch: ( 49) (  717/ 3165)
Epoch: ( 49) (  718/ 3165)
Epoch: ( 49) (  719/ 3165)
Epoch: ( 49) (  720/ 3165)
Epoch: ( 49) (  721/ 3165)
Epoch: ( 49) (  722/ 3165)
Epoch: ( 49) (  723/ 3165)
Epoch: ( 49) (  724/ 3165)
Epoch: ( 49) (  725/ 3165)
Epoch: ( 49) (  726/ 3165)
Epoch: ( 49) (  727/ 3165)
Epoch: ( 49) (  728/ 3165)
Epoch: ( 49) (  729/ 3165)
Epoch: ( 49) (  730/ 3165)
Epoch: ( 49) (  731/ 3165)
Epoch: ( 49) (  732/ 3165)
Epoch: ( 49) (  733/ 3165)
Epoch: ( 49) (  734/ 3165)
Epoch: ( 49) (  735/ 3165)
Epoch: ( 49) (  736/ 3165)
Epoch: ( 49) (  737/ 3165)
Epoch: ( 49) (  738/ 3165)
Epoch: ( 49) (  739/ 3165)
Epoch: ( 49) (  740/ 3165)
Epoch: ( 49) (  741/ 3165)
Epoch: ( 49) (  742/ 3165)
Epoch: ( 49) (  743/ 3165)
Epoch: ( 49) (  744/ 3165)
Epoch: ( 49) (  745/ 3165)
Epoch: ( 49) (  746/ 3165)
Epoch: ( 49) (  747/ 3165)
Epoch: ( 49) (  748/ 3165)
Epoch: ( 49) (  749/ 3165)
Epoch: ( 49) (  750/ 3165)
Epoch: ( 49) (  751/ 3165)
E

Epoch: ( 49) (  815/ 3165)
Epoch: ( 49) (  816/ 3165)
Epoch: ( 49) (  817/ 3165)
Epoch: ( 49) (  818/ 3165)
Epoch: ( 49) (  819/ 3165)
Epoch: ( 49) (  820/ 3165)
Epoch: ( 49) (  821/ 3165)
Epoch: ( 49) (  822/ 3165)
Epoch: ( 49) (  823/ 3165)
Epoch: ( 49) (  824/ 3165)
Epoch: ( 49) (  825/ 3165)
Epoch: ( 49) (  826/ 3165)
Epoch: ( 49) (  827/ 3165)
Epoch: ( 49) (  828/ 3165)
Epoch: ( 49) (  829/ 3165)
Epoch: ( 49) (  830/ 3165)
Epoch: ( 49) (  831/ 3165)
Epoch: ( 49) (  832/ 3165)
Epoch: ( 49) (  833/ 3165)
Epoch: ( 49) (  834/ 3165)
Epoch: ( 49) (  835/ 3165)
Epoch: ( 49) (  836/ 3165)
Epoch: ( 49) (  837/ 3165)
Epoch: ( 49) (  838/ 3165)
Epoch: ( 49) (  839/ 3165)
Epoch: ( 49) (  840/ 3165)
Epoch: ( 49) (  841/ 3165)
Epoch: ( 49) (  842/ 3165)
Epoch: ( 49) (  843/ 3165)
Epoch: ( 49) (  844/ 3165)
Epoch: ( 49) (  845/ 3165)
Epoch: ( 49) (  846/ 3165)
Epoch: ( 49) (  847/ 3165)
Epoch: ( 49) (  848/ 3165)
Epoch: ( 49) (  849/ 3165)
Epoch: ( 49) (  850/ 3165)
Epoch: ( 49) (  851/ 3165)
E

Model saved in file: /content/drive/My Drive/GP/checkpoints/celeba_dcgan/Epoch_(49)_(915of3165).ckpt
Epoch: ( 49) (  916/ 3165)
Epoch: ( 49) (  917/ 3165)
Epoch: ( 49) (  918/ 3165)
Epoch: ( 49) (  919/ 3165)
Epoch: ( 49) (  920/ 3165)
Epoch: ( 49) (  921/ 3165)
Epoch: ( 49) (  922/ 3165)
Epoch: ( 49) (  923/ 3165)
Epoch: ( 49) (  924/ 3165)
Epoch: ( 49) (  925/ 3165)
Epoch: ( 49) (  926/ 3165)
Epoch: ( 49) (  927/ 3165)
Epoch: ( 49) (  928/ 3165)
Epoch: ( 49) (  929/ 3165)
Epoch: ( 49) (  930/ 3165)
Epoch: ( 49) (  931/ 3165)
Epoch: ( 49) (  932/ 3165)
Epoch: ( 49) (  933/ 3165)
Epoch: ( 49) (  934/ 3165)
Epoch: ( 49) (  935/ 3165)
Epoch: ( 49) (  936/ 3165)
Epoch: ( 49) (  937/ 3165)
Epoch: ( 49) (  938/ 3165)
Epoch: ( 49) (  939/ 3165)
Epoch: ( 49) (  940/ 3165)
Epoch: ( 49) (  941/ 3165)
Epoch: ( 49) (  942/ 3165)
Epoch: ( 49) (  943/ 3165)
Epoch: ( 49) (  944/ 3165)
Epoch: ( 49) (  945/ 3165)
Epoch: ( 49) (  946/ 3165)
Epoch: ( 49) (  947/ 3165)
Epoch: ( 49) (  948/ 3165)
Epoch: (

Epoch: ( 49) ( 1015/ 3165)
Epoch: ( 49) ( 1016/ 3165)
Epoch: ( 49) ( 1017/ 3165)
Epoch: ( 49) ( 1018/ 3165)
Epoch: ( 49) ( 1019/ 3165)
Epoch: ( 49) ( 1020/ 3165)
Epoch: ( 49) ( 1021/ 3165)
Epoch: ( 49) ( 1022/ 3165)
Epoch: ( 49) ( 1023/ 3165)
Epoch: ( 49) ( 1024/ 3165)
Epoch: ( 49) ( 1025/ 3165)
Epoch: ( 49) ( 1026/ 3165)
Epoch: ( 49) ( 1027/ 3165)
Epoch: ( 49) ( 1028/ 3165)
Epoch: ( 49) ( 1029/ 3165)
Epoch: ( 49) ( 1030/ 3165)
Epoch: ( 49) ( 1031/ 3165)
Epoch: ( 49) ( 1032/ 3165)
Epoch: ( 49) ( 1033/ 3165)
Epoch: ( 49) ( 1034/ 3165)
Epoch: ( 49) ( 1035/ 3165)
Epoch: ( 49) ( 1036/ 3165)
Epoch: ( 49) ( 1037/ 3165)
Epoch: ( 49) ( 1038/ 3165)
Epoch: ( 49) ( 1039/ 3165)
Epoch: ( 49) ( 1040/ 3165)
Epoch: ( 49) ( 1041/ 3165)
Epoch: ( 49) ( 1042/ 3165)


In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models

def to_range(images, min_value=0.0, max_value=1.0, dtype=None):
    """
    transform images from [-1.0, 1.0] to [min_value, max_value] of dtype
    """
    assert \
        np.min(images) >= -1.0 - 1e-5 and np.max(images) <= 1.0 + 1e-5 \
        and (images.dtype == np.float32 or images.dtype == np.float64), \
        'The input images should be float64(32) and in the range of [-1.0, 1.0]!'
    if dtype is None:
        dtype = images.dtype
    return ((images + 1.) / 2. * (max_value - min_value) + min_value).astype(dtype)


def batchimwrite(image, path, cnt, prob):
    """ save an [-1.0, 1.0] image """

    for i in range(image.shape[0]):
        image = np.array(image, copy=True)

        if (cnt < 200000) :
            imageio.imwrite("%s_%d.jpg" % (path, cnt) , to_range(image[i, :, :, :], 0, 255, np.uint8))
            cnt += 1
            print(cnt,prob[i])

    return cnt

""" param """
epoch = 50
batch_size = 100
lr = 0.0002
z_dim = 100
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]
tf.reset_default_graph()

""" graphs """
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator

    ''' graph '''
    # inputs
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, training=False ,reuse=False)
    f_logit = discriminator(fake,training=False ,reuse=False)


""" train """
''' init '''
# session
sess = utils.session()
# saver
saver = tf.train.Saver()

''' initialization '''
ckpt_dir = '/content/drive/My Drive/GP/checkpoints/celeba_dcgan/Epoch_(5)_(2175of3165).ckpt'
saver.restore(sess, ckpt_dir)
sess.run(tf.global_variables_initializer())
saver.restore(sess, ckpt_dir)

try:
    
    cnt = 185000
    while(True):
        if(cnt >= 200000):
          break
        z_ipt = np.random.normal(size=[batch_size, z_dim])
        img = sess.run(fake, feed_dict={z: z_ipt})
        prob = sess.run(f_logit,{z: z_ipt})
        loss = tf.reduce_mean(prob)
        l = sess.run(loss)
        print(l)
        save_dir = '/content/drive/My Drive/GP/results/DCGAN(7)(15K)'
        utils.mkdir(save_dir + '/')
        cnt = batchimwrite(img, '%s/img' % (save_dir), cnt, prob)


except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()

Streaming output truncated to the last 5000 lines.
195052 [-2.6909645]
195053 [-1.7852297]
195054 [-5.0825195]
195055 [-3.3907292]
195056 [-1.8869313]
195057 [-4.316822]
195058 [-1.6924137]
195059 [-4.2306876]
195060 [-2.121447]
195061 [-4.959436]
195062 [-3.8496401]
195063 [0.01437439]
195064 [-2.4281318]
195065 [-3.3693905]
195066 [-2.6280172]
195067 [-2.2884688]
195068 [-3.4917343]
195069 [-3.141149]
195070 [-1.3984097]
195071 [-2.05662]
195072 [-3.8042312]
195073 [-2.9588778]
195074 [-1.593835]
195075 [-2.000682]
195076 [-3.327882]
195077 [-1.6352987]
195078 [-2.823477]
195079 [-3.402309]
195080 [-0.525364]
195081 [-3.2091925]
195082 [-5.016109]
195083 [-3.4859095]
195084 [-4.4372926]
195085 [-4.364926]
195086 [-5.5341177]
195087 [-2.3896797]
195088 [-2.9403477]
195089 [-3.0537026]
195090 [-3.3697278]
195091 [-1.3975447]
195092 [-4.3806834]
195093 [-2.1755736]
195094 [-7.6807256]
195095 [-0.9080355]
195096 [-3.5544093]
195097 [-1.7930847]
195098 [-5.7738724]
195099 [-3.6923351]
195

In [ ]:
!zip -r '/content/drive/My Drive/GP/results/images6.zip' '/content/drive/My Drive/GP/results/celeba_dcgan6'

Streaming output truncated to the last 5000 lines.
	zip warning: could not open for reading: content/drive/My Drive/GP/results/celeba_dcgan6/img_68334.jpg
  adding: content/drive/My Drive/GP/results/celeba_dcgan6/img_68335.jpg
zip warning: No such file or directory
	zip warning: could not open for reading: content/drive/My Drive/GP/results/celeba_dcgan6/img_68335.jpg
  adding: content/drive/My Drive/GP/results/celeba_dcgan6/img_68336.jpg
zip warning: No such file or directory
	zip warning: could not open for reading: content/drive/My Drive/GP/results/celeba_dcgan6/img_68336.jpg
  adding: content/drive/My Drive/GP/results/celeba_dcgan6/img_68337.jpg
zip warning: No such file or directory
	zip warning: could not open for reading: content/drive/My Drive/GP/results/celeba_dcgan6/img_68337.jpg
  adding: content/drive/My Drive/GP/results/celeba_dcgan6/img_68338.jpg
zip warning: No such file or directory
	zip warning: could not open for reading: content/drive/My Drive/GP/results/celeba_dcgan6